In [20]:
%config Completer.use_jedi = False

In [21]:
import os
from tqdm.notebook import tqdm
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations
import torch

from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [22]:
import sys
sys.path.append('../')

from utils.general import *
import utils.dataload as d
from models import model_selector
from utils.data_augmentation import data_augmentation_selector
from medpy.metric.binary import hd, dc, jc, assd

from utils.neural import *
from utils.datasets import *
from utils.metrics import *

In [23]:
def find_path(directory, filename):
    for path in Path(directory).rglob(filename):
        return path

In [24]:
value_ranges = [0, 0.25, 0.5, 0.75, 1]
values_desc = ["awful", "average", "good", "excellent"]

for i in range(4):
    print(f"{value_ranges[i]} - {value_ranges[i+1]}: {values_desc[i]}")

0 - 0.25: awful
0.25 - 0.5: average
0.5 - 0.75: good
0.75 - 1: excellent


In [45]:
os.path.splitext(os.path.basename("check.pt"))[0]

'check'

In [40]:
os.path.basename("check.pt")

'check.pt'

In [27]:
model = model_selector(
    "segmentation", "resnet34_unet_imagenet_encoder_scse_hypercols", num_classes=4, from_swa=False,
    in_channels=3, devices=[0], checkpoint="../checks/ACDC/n1_100.pt"
)

_, _, val_aug = data_augmentation_selector(
    "none", 224, 224, "padd"
)


--- Frosted pretrained backbone! ---
Model total number of parameters: 35749488
Loaded model from checkpoint: ../checks/ACDC/n1_100.pt
Using None Data Augmentation
Padding masks!
Padding masks!


In [28]:
batch_size = 1
add_depth = True
normalization = "standardize"

train_dataset = ALLMMsDataset3D(
    transform=val_aug, img_transform=[],
    add_depth=add_depth, normalization=normalization, data_relative_path="../../mnms_da",
    vendor=["A"]  # Solo queremos comparar con mismo fabricante -> Siemens == Vendor A
)

allmms_loader =  DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True,
    drop_last=False, collate_fn=train_dataset.simple_collate
)

In [29]:
train_dataset.df

,External code,VendorName,Vendor,Centre,ED,ES,Partition,Labeled
0,A0S9V9,Siemens,A,1,0,9,Training,True
1,A1D9Z7,Siemens,A,1,22,11,Training,True
2,A1E9Q1,Siemens,A,1,0,9,Training,True
3,A2C0I1,Siemens,A,1,0,7,Training,True
4,A2L1N6,Siemens,A,1,0,12,Testing,False
...,...,...,...,...,...,...,...,...
90,Q3R9W7,Siemens,A,1,0,10,Training,True
91,R4Y1Z9,Siemens,A,1,24,7,Training,True
92,S1S3Z7,Siemens,A,1,0,9,Training,True
93,T2T9Z9,Siemens,A,1,0,13,Training,True


In [30]:
def map_mask_classes(mask, classes_map):
    """

    Args:
        mask: (np.array) Mask Array to map (height, width)
        classes_map: (dict) Mapping between classes. E.g.  {0:0, 1:3, 2:2, 3:1 ,4:4}

    Returns: (np.array) Mapped mask array

    """
    res = np.zeros_like(mask).astype(mask.dtype)
    for value in np.unique(mask):
        if value not in classes_map:
            assert False, f"Please specify all class maps. {value} not in {classes_map}"
        res += np.where(mask == value, classes_map[value], 0).astype(mask.dtype)
    return res

In [31]:
model.eval()

metrics = {
    'img_id':[], 'phase':[],
    'iou_RV':[], 'dice_RV':[], 'hd_RV':[], 'assd_RV': [], 
    'iou_MYO':[], 'dice_MYO':[], 'hd_MYO':[], 'assd_MYO': [],
    'iou_LV':[], 'dice_LV':[], 'hd_LV':[], 'assd_LV': [],
}
plot_per_range = [50,50,50,50] # ["awful", "average", "good", "excellent"]
preds_dir = "CrossDatabase_v2/ACDCD2ALLMMS3D"
os.makedirs(preds_dir, exist_ok=True)

# MMS -> class_to_cat = {1: "LV", 2: "MYO", 3: "RV"}
# Original ACDC -> class_to_cat = {1: "RV", 2: "MYO", 3: "LV"}
map_classes = {0: 0, 1: 3, 2: 2, 3: 1}
class_to_cat = {1: "RV", 2: "MYO", 3: "LV"}
mask_reshape_method = "padd"
include_background = False

with torch.no_grad():
    for batch_indx, batch in enumerate(tqdm(allmms_loader)):
        img_id = batch["external_code"]
        partition = batch["partition"]
        init_shape = batch["initial_shape"]
        
        ed_vol = batch["ed_volume"].squeeze().cuda()
        es_vol = batch["es_volume"].squeeze().cuda()
        
        original_ed = batch["original_ed"]
        original_es = batch["original_es"]
        
        original_ed_mask = batch["original_ed_mask"]
        original_es_mask = batch["original_es_mask"]
        
        mask_affine = batch["affine"]
        mask_header = batch["header"]
        
        save_dir = os.path.join(preds_dir, partition, "Labeled" if partition=="Training" else "", img_id)
        os.makedirs(save_dir, exist_ok=True)
        
        # ---- ED CASE ----
        
        prob_preds = model(ed_vol)
        pred_mask = convert_multiclass_mask(prob_preds).data.cpu().numpy()
        pred_mask = pred_mask.astype(np.uint8)
        pred_mask = map_mask_classes(pred_mask, map_classes)
        pred_mask = reshape_volume(pred_mask, init_shape[:2], mask_reshape_method)
        pred_mask = pred_mask.transpose(1,2,0)
        
        for current_class in range(len(map_classes)):

            if not include_background and current_class == 0:
                continue

            y_true = np.where(original_ed_mask == current_class, 1, 0).astype(np.int32)
            y_pred = np.where(pred_mask == current_class, 1, 0).astype(np.int32)
            class_str = class_to_cat[current_class]

            jc_score = jaccard_coef(y_true, y_pred)
            dc_score = dice_coef(y_true, y_pred)
            hd_score = secure_hd(y_true, y_pred)
            assd_score = secure_assd(y_true, y_pred)

            metrics[f'iou_{class_str}'].append(jc_score)
            metrics[f'dice_{class_str}'].append(dc_score)
            metrics[f'hd_{class_str}'].append(hd_score)
            metrics[f'assd_{class_str}'].append(assd_score)

        metrics[f'img_id'].append(img_id)
        metrics[f'phase'].append("ED")
    
        pred_name = f"{img_id}_ED.nii.gz"
        d.save_nii(os.path.join(save_dir, pred_name), pred_mask, mask_affine, mask_header)
        
        # ---- ES CASE ----
        
        prob_preds = model(es_vol)
        pred_mask = convert_multiclass_mask(prob_preds).data.cpu().numpy()
        pred_mask = pred_mask.astype(np.uint8)
        pred_mask = map_mask_classes(pred_mask, map_classes)
        pred_mask = reshape_volume(pred_mask, init_shape[:2], mask_reshape_method)
        pred_mask = pred_mask.transpose(1,2,0)
        
        for current_class in range(len(map_classes)):

            if not include_background and current_class == 0:
                continue

            y_true = np.where(original_es_mask == current_class, 1, 0).astype(np.int32)
            y_pred = np.where(pred_mask == current_class, 1, 0).astype(np.int32)
            class_str = class_to_cat[current_class]

            jc_score = jaccard_coef(y_true, y_pred)
            dc_score = dice_coef(y_true, y_pred)
            hd_score = secure_hd(y_true, y_pred)
            assd_score = secure_assd(y_true, y_pred)

            metrics[f'iou_{class_str}'].append(jc_score)
            metrics[f'dice_{class_str}'].append(dc_score)
            metrics[f'hd_{class_str}'].append(hd_score)
            metrics[f'assd_{class_str}'].append(assd_score)

        metrics[f'img_id'].append(img_id)
        metrics[f'phase'].append("ES")
    
        pred_name = f"{img_id}_ES.nii.gz"
        d.save_nii(os.path.join(save_dir, pred_name), pred_mask, mask_affine, mask_header)

  0%|          | 0/95 [00:00<?, ?it/s]

In [32]:
df = pd.DataFrame(metrics)
df.head()

,img_id,phase,iou_RV,dice_RV,hd_RV,assd_RV,iou_MYO,dice_MYO,hd_MYO,assd_MYO,iou_LV,dice_LV,hd_LV,assd_LV
0,A0S9V9,ED,0.898427,0.946496,5.000000,0.239037,0.609531,0.757402,10.049876,0.438209,0.830003,0.907106,35.114100,0.364235
1,A0S9V9,ES,0.893836,0.943943,4.690416,0.233343,0.656237,0.792443,7.000000,0.337312,0.784602,0.879302,5.830952,0.277703
2,A1D9Z7,ED,0.809625,0.894799,5.000000,0.559468,0.692625,0.818404,12.083046,0.635774,0.781592,0.877408,8.602325,0.461106
3,A1D9Z7,ES,0.556254,0.714863,14.177447,1.391500,0.619211,0.764831,10.677078,0.914958,0.681189,0.810366,11.224972,0.625159
4,A1E9Q1,ED,0.916648,0.956512,5.099020,0.141119,0.610340,0.758026,6.000000,0.397472,0.884548,0.938737,5.099020,0.162448


## Get metrics by replacing infinite distance values with max value

In [33]:
print(f"Mean IOU RV: {df['iou_RV'].mean()}")
print(f"Mean IOU LV: {df['iou_LV'].mean()}")
print(f"Mean IOU MYO: {df['iou_MYO'].mean()}")

print("--------------")

print(f"Mean DICE RV: {df['dice_RV'].mean()}")
print(f"Mean DICE LV: {df['dice_LV'].mean()}")
print(f"Mean DICE MYO: {df['dice_MYO'].mean()}")

print("--------------")

print(f"Mean Hausdorff RV: {df['hd_RV'].mean()}")
print(f"Mean Hausdorff LV: {df['hd_LV'].mean()}")
print(f"Mean Hausdorff MYO: {df['hd_MYO'].mean()}")

print("--------------")

print(f"Mean ASSD RV: {df['assd_RV'].mean()}")
print(f"Mean ASSD LV: {df['assd_LV'].mean()}")
print(f"Mean ASSD MYO: {df['assd_MYO'].mean()}")

Mean IOU RV: 0.80881525207711
Mean IOU LV: 0.7385841102720277
Mean IOU MYO: 0.6576854756184404
--------------
Mean DICE RV: 0.8887979457784803
Mean DICE LV: 0.8419029968319072
Mean DICE MYO: 0.7919082956366139
--------------
Mean Hausdorff RV: 15.36982163099437
Mean Hausdorff LV: 11.674009025036435
Mean Hausdorff MYO: 17.19066310565214
--------------
Mean ASSD RV: 0.6690749543364946
Mean ASSD LV: 0.5006301037926716
Mean ASSD MYO: 0.5418732081833872


In [34]:
min_hausdorff_lv = df["hd_LV"].min()
print(f"min_hausdorff_lv: {min_hausdorff_lv}")
min_hausdorff_rv = df["hd_RV"].min()
print(f"min_hausdorff_rv: {min_hausdorff_rv}")
min_hausdorff_myo = df["hd_MYO"].min()
print(f"min_hausdorff_myo: {min_hausdorff_myo}")

min_assd_lv = df["assd_LV"].min()
print(f"min_assd_lv: {min_assd_lv}")
min_assd_rv = df["assd_RV"].min()
print(f"min_assd_rv: {min_assd_rv}")
min_assd_myo = df["assd_MYO"].min()
print(f"min_assd_myo: {min_assd_myo}")

min_hausdorff_lv: 2.23606797749979
min_hausdorff_rv: 1.4142135623730951
min_hausdorff_myo: 2.23606797749979
min_assd_lv: 0.1423923147440977
min_assd_rv: 0.09066776705287188
min_assd_myo: 0.17965383192414908


In [35]:
max_hausdorff_lv = df["hd_LV"].max()
max_hausdorff_rv = df["hd_RV"].max()
max_hausdorff_myo = df["hd_MYO"].max()

max_assd_lv = df["assd_LV"].max()
max_assd_rv = df["assd_RV"].max()
max_assd_myo = df["assd_MYO"].max()

In [36]:
print(f"Mean IOU RV: {df['iou_RV'].mean()}")
print(f"Mean IOU LV: {df['iou_LV'].mean()}")
print(f"Mean IOU MYO: {df['iou_MYO'].mean()}")

print("--------------")

print(f"Mean DICE RV: {df['dice_RV'].mean()}")
print(f"Mean DICE LV: {df['dice_LV'].mean()}")
print(f"Mean DICE MYO: {df['dice_MYO'].mean()}")

print("--------------")

print(f"Mean Hausdorff RV: {df['hd_RV'].mean()}")
print(f"Mean Hausdorff LV: {df['hd_LV'].mean()}")
print(f"Mean Hausdorff MYO: {df['hd_MYO'].mean()}")

print("--------------")

print(f"Mean ASSD RV: {df['assd_RV'].mean()}")
print(f"Mean ASSD LV: {df['assd_LV'].mean()}")
print(f"Mean ASSD MYO: {df['assd_MYO'].mean()}")

Mean IOU RV: 0.80881525207711
Mean IOU LV: 0.7385841102720277
Mean IOU MYO: 0.6576854756184404
--------------
Mean DICE RV: 0.8887979457784803
Mean DICE LV: 0.8419029968319072
Mean DICE MYO: 0.7919082956366139
--------------
Mean Hausdorff RV: 15.36982163099437
Mean Hausdorff LV: 11.674009025036435
Mean Hausdorff MYO: 17.19066310565214
--------------
Mean ASSD RV: 0.6690749543364946
Mean ASSD LV: 0.5006301037926716
Mean ASSD MYO: 0.5418732081833872


In [37]:
df.groupby("phase").mean()

,iou_RV,dice_RV,hd_RV,assd_RV,iou_MYO,dice_MYO,hd_MYO,assd_MYO,iou_LV,dice_LV,hd_LV,assd_LV
phase,,,,,,,,,,,,
ED,0.873114,0.931355,13.732099,0.468584,0.662797,0.795782,17.515205,0.499268,0.799854,0.885875,11.923436,0.411896
ES,0.744517,0.846241,17.007544,0.869566,0.652574,0.788034,16.866121,0.584479,0.677314,0.797931,11.424582,0.589364


In [17]:
df.to_csv(os.path.join(preds_dir, "results.csv"), index=False)